<a href="https://colab.research.google.com/github/Toroi0610/Baseball/blob/master/Markov_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マルコフ過程に基づくシーズン得点数の推定

マルコフ過程とは未来の状態が現在の状態のみから確率的に決まるような確率過程のことを言います。式では次のようになります．

\begin{equation}
    S_{t+1} = P(t)S_{t}\tag{1}
\end{equation}

ここで$S_t$は時刻$t$において状態$j$にある確率を$j$番目の要素として持つベクトルで$j=1, 2, ..., n$です．そのため，

\begin{equation}
    \sum_{j=1}^nS_t(j) = 1\tag{2}
\end{equation}

を時刻によらず満たします．

次に，$P(t)$は$n\times n$の推移確率行列です．$P(t)$の$ij$成分$P_{ij}(t)$は次のように定義されています．

\begin{equation}
    P_{ij}(t) = P(X_{t+1}=j|X_{t}=i, t)\tag{3}
\end{equation}
式の右辺は時刻$t$における状態$i$が，時刻$t+1$で状態$j$にある確率を表しています．つまり式(1)は時刻$t$での各状態の実現確率から時刻$t+1$における各状態の実現確率を計算しています．



## Reference
Bukiet, Bruce, Elliotte Rusty Harold, and José Luis Palacios. "A Markov Chain Approach to Baseball." Operations Research 45, no. 1 (1997): 14-23. http://www.jstor.org/stable/171922.